In [18]:
import argparse
import torch
import numpy as np
import os
import time
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import scipy.sparse as ss
import torch.optim as optim
import matplotlib.pyplot as plt
# from dgtv.dgtv import *
import pickle
import logging
import sys
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler



In [2]:
# !python train_gtv.py --batch 1 --lr 1e-3 --umin 0.01 --umax 100 -m model/GTV.pkl --epoch 5 --train dataset/dataset_structure --width 36

In [19]:
class OPT:
    def __init__(
        self,
        batch_size=100,
        width=36,
        connectivity="8",
        channels=3,
        u_max=100,
        u_min=10,
        lr=1e-4,
        momentum=0.99,
        ver=None,
        train="gauss_batch",
        cuda=False,
        logger=None,
        legacy=False
    ):
        self.batch_size = batch_size
        self.legacy = legacy
        self.width = width
        self.edges = 0
        self.nodes = width ** 2
        self.I = None
        self.pairs = None
        self.H = None
        self.connectivity = connectivity
        self.channels = channels
        self.lr = lr
        self.momentum = momentum
        self.u_max = u_max
        self.u_min = u_min
        self.ver = ver
        self.D = None
        self.train = train
        self.cuda = cuda
        if cuda:
            self.dtype = torch.cuda.FloatTensor
        else:
            self.dtype = torch.FloatTensor
        self.logger = logger

    def _print(self):
        self.logger.info(
            "batch_size = {0}, width = {1}, channels = {2}, u_min = {3}, u_max = {4}, lr = {5}, momentum = {6}".format(
                self.batch_size,
                self.width,
                self.channels,
                self.u_min,
                self.u_max,
                self.lr,
                self.momentum,
            )
        )

In [37]:

def connected_adjacency(image, connect="8", patch_size=(1, 1)):
    """
    Construct 8-connected pixels base graph (0 for not connected, 1 for connected)
    """
    r, c = image.shape[:2]
    r = int(r / patch_size[0])
    c = int(c / patch_size[1])

    # if connect == "4":
    if connect == "4":
        # constructed from 2 diagonals above the main diagonal
        d1 = np.tile(np.append(np.ones(c - 1), [0]), r)[:-1]
        print(d1)
        
        print(d1.shape)
        
        d2 = np.ones(c * (r - 1))
        print(d2)
        
        print(d2.shape)
        
        upper_diags = ss.diags([d1, d2], [1, c])
        print(upper_diags)
        print(upper_diags.shape)
        print(upper_diags + upper_diags.T)
        
        return upper_diags + upper_diags.T
    
    elif connect == "8":
        # constructed from 4 diagonals above the main diagonal
        d1 = np.tile(np.append(np.ones(c - 1), [0]), r)[:-1]
        # print(d1)
        # print(d1.shape)

        d2 = np.append([0], d1[: c * (r - 1)])
        # print(d2)
        # print(d2.shape)

        d3 = np.ones(c * (r - 1))
        # print(d3)
        # print(d3.shape)

        d4 = d2[1:-1]
        # print(d4)
        # print(d4.shape)

        upper_diags = ss.diags([d1, d2, d3, d4], [1, c - 1, c, c + 1])
        # print(upper_diags)
        # print(upper_diags.shape)
        # print(upper_diags + upper_diags.T)
        return upper_diags + upper_diags.T

width = 3
pixel_indices = [i for i in range(width * width)]
pixel_indices = np.reshape(pixel_indices, (width, width))
A = connected_adjacency(pixel_indices,"8",(1,1))
A_pair = np.asarray(np.where(A.toarray() == 1)).T
A_pair = np.unique(np.sort(A_pair, axis=1), axis=0)
print(A_pair)
print(A_pair.shape)

[[0 1]
 [0 3]
 [0 4]
 [1 2]
 [1 3]
 [1 4]
 [1 5]
 [2 4]
 [2 5]
 [3 4]
 [3 6]
 [3 7]
 [4 5]
 [4 6]
 [4 7]
 [4 8]
 [5 7]
 [5 8]
 [6 7]
 [7 8]]
(20, 2)


In [41]:
def supporting_matrix(opt):
    dtype = opt.dtype
    cuda = opt.cuda
    width = opt.width

    pixel_indices = [i for i in range(width * width)]
    pixel_indices = np.reshape(pixel_indices, (width, width))
    # tạo một danh sách pixel_indices chứa các số nguyên từ 0 đến (width * width) - 1. 
    #  chuyển nó thành một ma trận 2D với kích thước (width, width) bằng cách sử dụng np.reshape.
    A = connected_adjacency(pixel_indices, connect=opt.connectivity)
    # gọi hàm connected_adjacency với đối số pixel_indices để tạo ma trận kết nối (adjacency matrix) A. 
    # Giá trị connect được lấy từ opt để xác định cách kết nối giữa các pixel.
    A_pair = np.asarray(np.where(A.toarray() == 1)).T
    A_pair = np.unique(np.sort(A_pair, axis=1), axis=0)
    #tìm các cặp pixel kết nối bằng cách xác định những vị trí trong ma trận A có giá trị bằng 1. 
    # Sau đó, sắp xếp các cặp này và loại bỏ các cặp trùng lặp để có được A_pair, một danh sách các cặp pixel kết nối.
    
    opt.edges = A_pair.shape[0]
    #đặt thuộc tính edges trong đối tượng opt bằng số lượng cặp pixel kết nối trong A_pair.

    
    H_dim0 = opt.edges
    H_dim1 = width ** 2
    #tính toán kích thước của ma trận H dựa trên số lượng cặp pixel kết nối và kích thước hình ảnh.

    I = torch.eye(width ** 2, width ** 2).type(dtype)
    #tạo ma trận đơn vị I có kích thước (width ** 2, width ** 2) sử dụng PyTorch, với kiểu dữ liệu dtype.
    A = torch.zeros(width ** 2, width ** 2).type(dtype)
    H = torch.zeros(H_dim0, H_dim1).type(dtype)
    #tạo ma trận A và H tương ứng
    for e, p in enumerate(A_pair):
        H[e, p[0]] = 1
        H[e, p[1]] = -1
        A[p[0], p[1]] = 1
    #xây dựng các ma trận A và H bằng cách gán giá trị 1 và -1 vào các vị trí tương ứng trong ma trận H.
    #Đồng thời, chúng ta cập nhật các giá trị trong ma trận A để thể hiện sự kết nối giữa các pixel.

    opt.I = I
    opt.pairs = A_pair
    opt.H = H
    opt.connectivity_full = A.requires_grad_(True)
    opt.connectivity_idx = torch.where(A > 0)
    #gán các ma trận và danh sách I, pairs, H, connectivity_full, và connectivity_idx vào thuộc tính của đối tượng opt. 
    # Đồng thời, chúng ta đánh dấu ma trận A cho phép tính đạo hàm (requires_grad_).

    for e, p in enumerate(A_pair):
        A_temp = A.clone()
        A_temp[p[1], p[0]] = 1
        A = A_temp
    #duyệt ktra một lần nữa
    
    # opt.logger.info("OPT created on cuda: {0} {1}".format(cuda, dtype))
    if opt.logger:
        opt.logger.info("OPT created on cuda: {0} {1}".format(cuda, dtype))
    else:
        print("OPT created on cuda: {0} {1}".format(cuda, dtype))


    # print(A_pair)
    # print(A_pair.shape)



In [42]:
opt = OPT(
    batch_size=32,
    channels=3,
    lr=1e-4,
    momentum=0.9,
    u_max=1000,
    u_min=0.0001,
    width=3,
    cuda=True if torch.cuda.is_available() else False

)
supporting_matrix(opt)

OPT created on cuda: False <class 'torch.FloatTensor'>
[[0 1]
 [0 3]
 [0 4]
 [1 2]
 [1 3]
 [1 4]
 [1 5]
 [2 4]
 [2 5]
 [3 4]
 [3 6]
 [3 7]
 [4 5]
 [4 6]
 [4 7]
 [4 8]
 [5 7]
 [5 8]
 [6 7]
 [7 8]]
(20, 2)


In [5]:
def test(
    seed, model_name, cont=None, optim_name=None, subset=None, epoch=100, args=None
):
    cuda = True if torch.cuda.is_available() else False
    torch.autograd.set_detect_anomaly(True)
    opt.logger.info("CUDA: {0}".format(cuda))
    if cuda:
        dtype = torch.cuda.FloatTensor
        opt.logger.info(torch.cuda.get_device_name(0))
    else:
        dtype = torch.FloatTensor

    DST = "./"
    DST = ""
    PATH = os.path.join(DST, model_name)
    SAVEPATH = PATH.split(".")[-1]
    SAVEDIR = "".join(PATH.split(".")[:-1]) + "_"
    batch_size = opt.batch_size
    if not subset:
        _subset = ["10", "1", "7", "8", "9"]
        # _subset = ["1", "3", "5", "7", "9"]
        opt.logger.info("Train: {0}".format(_subset))
        subset = [i + "_" for i in _subset]
    else:
        subset = [i + "_" for i in subset]
    dataset = RENOIR_Dataset(
        img_dir=os.path.join(opt.train),
        transform=transforms.Compose(
            [standardize(normalize=False), ToTensor()]),
        subset=None,
    )
    opt.logger.info("Splitting patches...")
    patch_splitting(
        dataset=dataset, output_dst="tmp", patch_size=args.width, stride=args.width / 2
    )
    dataset = RENOIR_Dataset(
        img_dir=os.path.join("tmp", "patches"),
        transform=transforms.Compose(
            [standardize(normalize=False), ToTensor()]),
        subset=subset,
    )

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=4,
        drop_last=True,
    )

    supporting_matrix(opt)

In [ ]:


# Đoạn code để tạo một tập dữ liệu giả định từ GTV
# Đảm bảo rằng data_features và data_labels được xây dựng từ đặc trưng GTV và nhãn tương ứng.

# Xác định đặc trưng và nhãn từ dữ liệu GTV sử dụng supporting_matrix(opt)
opt = ...  # Đối tượng opt được sử dụng để gọi supporting_matrix(opt)
data_features = supporting_matrix(opt)  # Dùng hàm supporting_matrix để trích xuất đặc trưng
data_labels = [...]  # Nhãn tương ứng

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(data_features, data_labels, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Xây dựng mô hình KNN với K=3 (có thể điều chỉnh K theo nhu cầu)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Dự đoán nhãn của dữ liệu kiểm tra
y_pred = knn.predict(X_test)

# Đánh giá hiệu suất của mô hình KNN
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
